In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df = pd.read_csv('diabetes_cleaned_12-15-20.csv')

In [2]:
from sklearn.model_selection import train_test_split

X = df.drop(columns='readmit_30d')
y = df.readmit_30d
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [3]:
len(X_train.columns)

43

In [18]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

params = {'n_estimators':[100,200]}
estimator = GradientBoostingClassifier(random_state=5,verbose=1,class_weight='balanced')
roc_scorer = make_scorer(roc_auc_score)
scaler_X = StandardScaler().fit(X_train)
scaled_X = scaler_X.transform(X_train)
scaled_X = pd.DataFrame(scaled_X,index=X_train.index)

gscv = GridSearchCV(estimator=estimator,param_grid=params,scoring=roc_scorer,cv=5,verbose=1,n_jobs=3)
gscv.fit(scaled_X,y_train)

TypeError: __init__() got an unexpected keyword argument 'class_weight'

In [16]:
gscv.cv_results_

{'mean_fit_time': array([13.85038304, 23.13656793]),
 'std_fit_time': array([0.41406242, 5.5292048 ]),
 'mean_score_time': array([0.0587851, 0.0650775]),
 'std_score_time': array([0.00935017, 0.02447723]),
 'param_n_estimators': masked_array(data=[100, 200],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_estimators': 100}, {'n_estimators': 200}],
 'split0_test_score': array([0.50323727, 0.50403605]),
 'split1_test_score': array([0.50304102, 0.50483482]),
 'split2_test_score': array([0.50244098, 0.50428582]),
 'split3_test_score': array([0.50174672, 0.50259554]),
 'split4_test_score': array([0.50195029, 0.50335351]),
 'mean_test_score': array([0.50248329, 0.50382118]),
 'std_test_score': array([0.00058446, 0.00077594]),
 'rank_test_score': array([2, 1], dtype=int32)}

In [64]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

roc_scorer = make_scorer(roc_auc_score)
kf = KFold(n_splits=5,shuffle=True,random_state=5)
estimator = GradientBoostingClassifier(random_state=5,verbose=1)

major_indices = y_train[y_train==0].index
minor_indices = y_train[y_train==1].index
sampled_indices = []
for i in range(42,52):
    sampled_indices.append(resample(y_train[major_indices],replace=False,n_samples=5022,random_state=i).index)
    
scaler_X = StandardScaler().fit(X_train)
#scaler_y = StandardScaler().fit(y_train.to_numpy().reshape(-1,1))
scaled_X = scaler_X.transform(X_train)
#scaled_y = scaler_y.transform(y_train.to_numpy().reshape(-1,1))
scaled_X = pd.DataFrame(scaled_X,index=X_train.index)
#scaled_y = pd.Series(scaled_y.flatten(),index=y_train.index)
                       
params = {'n_estimators':[100,300,500],'max_depth':[2,3],'learning_rate':[0.03,0.06,0.1]}

results = []
for i in sampled_indices:
    X_balanced = pd.concat([scaled_X.loc[i],scaled_X.loc[minor_indices]])
    y_balanced = pd.concat([y_train[i],y_train[minor_indices]])
    selector = GridSearchCV(estimator=estimator, param_grid=params,cv=kf, scoring=roc_scorer, verbose=1,n_jobs=-1)
    selector = selector.fit(X_balanced, y_balanced)
    results.append(selector.cv_results_)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   46.3s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  1.7min finished


      Iter       Train Loss   Remaining Time 
         1           1.3828            3.94s
         2           1.3798            3.94s
         3           1.3770            3.99s
         4           1.3745            3.92s
         5           1.3722            3.92s
         6           1.3700            3.91s
         7           1.3681            3.88s
         8           1.3664            3.88s
         9           1.3644            3.85s
        10           1.3629            3.95s
        20           1.3501            3.89s
        30           1.3418            4.53s
        40           1.3356            4.24s
        50           1.3305            3.96s
        60           1.3265            3.72s
        70           1.3232            3.70s
        80           1.3204            3.51s
        90           1.3179            3.37s
       100           1.3156            3.30s
       200           1.2995            2.25s
       300           1.2892            1.45s
       40

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   41.9s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  1.6min finished


      Iter       Train Loss   Remaining Time 
         1           1.3832            7.97s
         2           1.3804            5.89s
         3           1.3779            5.21s
         4           1.3756            4.91s
         5           1.3734            4.66s
         6           1.3714            4.56s
         7           1.3695            4.45s
         8           1.3679            4.51s
         9           1.3662            4.41s
        10           1.3646            4.33s
        20           1.3530            4.28s
        30           1.3453            4.54s
        40           1.3392            4.16s
        50           1.3344            3.99s
        60           1.3301            3.81s
        70           1.3264            3.65s
        80           1.3234            3.48s
        90           1.3208            3.34s
       100           1.3185            3.27s
       200           1.3034            2.26s
       300           1.2942            1.46s
       40

...

      Iter       Train Loss   Remaining Time 
         1           1.3803            3.97s
         2           1.3755            3.94s
         3           1.3713            3.98s
         4           1.3679            4.18s
         5           1.3646            4.07s
         6           1.3620            4.23s
         7           1.3592            4.13s
         8           1.3566            4.10s
         9           1.3544            4.11s
        10           1.3522            4.09s
        20           1.3365            4.05s
        30           1.3264            3.88s
        40           1.3187            3.67s
        50           1.3130            3.46s
        60           1.3081            3.28s
        70           1.3041            3.22s
        80           1.3005            3.12s
        90           1.2978            2.98s
       100           1.2955            2.91s
       200           1.2775            2.08s
       300           1.2663            1.37s
       40

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   47.4s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  1.7min finished


      Iter       Train Loss   Remaining Time 
         1           1.3807            0.81s
         2           1.3762            0.79s
         3           1.3723            0.78s
         4           1.3690            0.76s
         5           1.3658            0.76s
         6           1.3630            0.76s
         7           1.3607            0.78s
         8           1.3583            0.77s
         9           1.3561            0.75s
        10           1.3542            0.74s
        20           1.3409            0.61s
        30           1.3321            0.56s
        40           1.3259            0.46s
        50           1.3212            0.38s
        60           1.3173            0.29s
        70           1.3140            0.22s
        80           1.3110            0.15s
        90           1.3085            0.07s
       100           1.3064            0.00s
Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   41.3s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  1.6min finished


      Iter       Train Loss   Remaining Time 
         1           1.3806            3.67s
         2           1.3760            3.81s
         3           1.3719            3.88s
         4           1.3680            3.83s
         5           1.3647            3.86s
         6           1.3618            3.73s
         7           1.3592            3.82s
         8           1.3567            3.90s
         9           1.3545            3.86s
        10           1.3524            3.93s
        20           1.3373            3.62s
        30           1.3270            3.60s
        40           1.3196            3.37s
        50           1.3138            3.25s
        60           1.3092            3.19s
        70           1.3054            3.08s
        80           1.3022            2.96s
        90           1.2989            2.85s
       100           1.2965            2.76s
       200           1.2772            2.02s
       300           1.2652            1.33s
       40

In [54]:
tmp = results[0]['mean_test_score']
for i in range(1,len(results)):
    tmp = np.concatenate([tmp,results[i]['mean_test_score']])
tmp2 = pd.DataFrame(tmp.reshape(50,18)).apply(lambda x: x.mean(),axis=0)
tmp2.index = results[0]['params']
tmp2.sort_values()


TypeError: unhashable type: 'dict'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1653, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'dict'


{'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 100}    0.598794
{'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 150}    0.601404
{'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100}    0.601681
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 200}     0.602077
{'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 100}    0.602793
{'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 100}     0.602857
{'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 200}    0.603067
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 150}     0.603649
{'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 150}    0.603765
{'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 150}    0.604224
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100}     0.604229
{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}     0.604446
{'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 200}    0.604658
{'learning_rate': 0.05, 'max_depth': 3, 'n_estimato

In [60]:
pd.DataFrame(tmp.reshape(50,18))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.594209,0.595495,0.595939,0.595939,0.599033,0.597986,0.595652,0.597384,0.602130,0.595786,0.598133,0.598334,0.595405,0.597657,0.599654,0.600377,0.602010,0.601396
1,0.594967,0.597453,0.600203,0.600157,0.601418,0.600467,0.600771,0.602742,0.601426,0.600688,0.600345,0.600881,0.599204,0.598963,0.600337,0.602526,0.598469,0.594979
2,0.601222,0.604661,0.606198,0.606394,0.609098,0.611985,0.606676,0.606658,0.610674,0.605276,0.609146,0.611299,0.608618,0.608060,0.608930,0.608946,0.610145,0.606793
3,0.598003,0.599966,0.602662,0.601090,0.606650,0.607451,0.602243,0.604271,0.607122,0.601049,0.605759,0.607393,0.607892,0.607495,0.605412,0.606464,0.605448,0.604459
4,0.604236,0.605426,0.604474,0.603640,0.606315,0.608026,0.606960,0.607738,0.607708,0.605011,0.608279,0.609440,0.609158,0.608742,0.611313,0.604331,0.605546,0.608234
5,0.600162,0.603219,0.604736,0.603736,0.606054,0.608199,0.607164,0.607301,0.609777,0.603629,0.606061,0.609895,0.609370,0.610878,0.610712,0.609133,0.606494,0.605534
6,0.599385,0.602155,0.602561,0.600340,0.602609,0.601295,0.602310,0.603843,0.602154,0.602561,0.603819,0.605287,0.603110,0.602820,0.603276,0.602371,0.603393,0.601519
7,0.597859,0.601953,0.602677,0.602887,0.604958,0.606384,0.604729,0.607793,0.611354,0.601176,0.606699,0.608294,0.606212,0.611064,0.611273,0.608527,0.608965,0.609167
8,0.591271,0.594700,0.597932,0.595371,0.597743,0.600510,0.594830,0.596100,0.596754,0.599461,0.600281,0.598688,0.598957,0.600101,0.598401,0.597057,0.593462,0.589666
9,0.602663,0.606789,0.607856,0.606370,0.612071,0.612861,0.609832,0.611946,0.611926,0.610016,0.612008,0.615694,0.611544,0.612974,0.613342,0.611091,0.611890,0.610228


In [63]:
tmp = results[0]['mean_test_score']
for i in range(1,len(results)):
    tmp = np.concatenate([tmp,results[i]['mean_test_score']])
tmp2 = pd.DataFrame(tmp.reshape(10,42)).apply(lambda x: x.std(),axis=0)
tmp2.index = results[0]['params']
tmp2.sort_values()


TypeError: unhashable type: 'dict'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1653, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'dict'


{'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 50}     0.003016
{'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 200}    0.003659
{'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 50}      0.003716
{'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 50}     0.003750
{'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100}    0.003813
{'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 100}     0.003847
{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50}      0.003986
{'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 100}    0.004012
{'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 150}    0.004177
{'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 250}    0.004243
{'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 150}     0.004398
{'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 150}    0.004441
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100}     0.004500
{'learning_rate': 0.05, 'max_depth': 2, 'n_estimato

In [65]:
tmp = results[0]['mean_test_score']
for i in range(1,len(results)):
    tmp = np.concatenate([tmp,results[i]['mean_test_score']])
tmp2 = pd.DataFrame(tmp.reshape(10,18)).apply(lambda x: x.mean(),axis=0)
tmp2.index = results[0]['params']
tmp2.sort_values()

TypeError: unhashable type: 'dict'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1653, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'dict'


{'learning_rate': 0.03, 'max_depth': 2, 'n_estimators': 100}    0.595196
{'learning_rate': 0.03, 'max_depth': 3, 'n_estimators': 100}    0.598567
{'learning_rate': 0.06, 'max_depth': 2, 'n_estimators': 100}    0.600123
{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 500}     0.600404
{'learning_rate': 0.03, 'max_depth': 2, 'n_estimators': 300}    0.601977
{'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 100}     0.602465
{'learning_rate': 0.06, 'max_depth': 3, 'n_estimators': 100}    0.602970
{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 300}     0.604256
{'learning_rate': 0.03, 'max_depth': 2, 'n_estimators': 500}    0.604605
{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}     0.604947
{'learning_rate': 0.03, 'max_depth': 3, 'n_estimators': 300}    0.605303
{'learning_rate': 0.06, 'max_depth': 2, 'n_estimators': 300}    0.605482
{'learning_rate': 0.06, 'max_depth': 3, 'n_estimators': 500}    0.605728
{'learning_rate': 0.06, 'max_depth': 3, 'n_estimato

In [88]:
#Upsampling
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

roc_scorer = make_scorer(roc_auc_score)
kf = KFold(n_splits=10,shuffle=True,random_state=5)
estimator = GradientBoostingClassifier(random_state=5,verbose=1)

major_indices = y_train[y_train==0].index
minor_indices = y_train[y_train==1].index
sampled_indices = resample(y_train[minor_indices],replace=True,n_samples=50954,random_state=5).index
X_balanced = pd.concat([X_train.loc[major_indices],X_train.loc[sampled_indices]])
y_balanced = pd.concat([y_train[major_indices],y_train[sampled_indices]])

scaler_X = StandardScaler().fit(X_balanced)
#scaler_y = StandardScaler().fit(y_train.to_numpy().reshape(-1,1))
scaled_X = scaler_X.transform(X_balanced)
#scaled_y = scaler_y.transform(y_train.to_numpy().reshape(-1,1))
scaled_X = pd.DataFrame(scaled_X,index=X_balanced.index)
#scaled_y = pd.Series(scaled_y.flatten(),index=y_train.index)
                       
params = {'n_estimators':[50,100,150,200,250,300,400,500],'max_depth':[2,3,4,5],'learning_rate':[0.03,0.06,0.1,.13],
         'max_features':[0.1,0.3,0.5],'loss':['deviance','exponential']}

selector = GridSearchCV(estimator=estimator, param_grid=params,cv=kf,scoring=roc_scorer,verbose=1,n_jobs=2)
selector = selector.fit(scaled_X, y_balanced)
print(selector.cv_results_)

Fitting 10 folds for each of 768 candidates, totalling 7680 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:  1.8min
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed: 14.5min
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed: 44.2min
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed: 109.4min
[Parallel(n_jobs=2)]: Done 1246 tasks      | elapsed: 186.6min
[Parallel(n_jobs=2)]: Done 1796 tasks      | elapsed: 276.7min
[Parallel(n_jobs=2)]: Done 2446 tasks      | elapsed: 379.7min
[Parallel(n_jobs=2)]: Done 3196 tasks      | elapsed: 510.0min


KeyboardInterrupt: 

In [90]:
dir(selector)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_is_fitted',
 '_estimator_type',
 '_format_results',
 '_get_param_names',
 '_get_tags',
 '_required_parameters',
 '_run_search',
 'classes_',
 'cv',
 'decision_function',
 'error_score',
 'estimator',
 'fit',
 'get_params',
 'iid',
 'inverse_transform',
 'multimetric_',
 'n_jobs',
 'param_grid',
 'pre_dispatch',
 'predict',
 'predict_log_proba',
 'predict_proba',
 'refit',
 'return_train_score',
 'score',
 'scoring',
 'set_params',
 'transform',
 'verbose']

In [72]:
selector.cv_results_['params']

[{'learning_rate': 0.03, 'max_depth': 2, 'n_estimators': 300},
 {'learning_rate': 0.03, 'max_depth': 2, 'n_estimators': 500},
 {'learning_rate': 0.03, 'max_depth': 3, 'n_estimators': 300},
 {'learning_rate': 0.03, 'max_depth': 3, 'n_estimators': 500},
 {'learning_rate': 0.06, 'max_depth': 2, 'n_estimators': 300},
 {'learning_rate': 0.06, 'max_depth': 2, 'n_estimators': 500},
 {'learning_rate': 0.06, 'max_depth': 3, 'n_estimators': 300},
 {'learning_rate': 0.06, 'max_depth': 3, 'n_estimators': 500},
 {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 300},
 {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 500},
 {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 300},
 {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 500}]

In [73]:
selector.cv_results_['mean_test_score']

array([0.60956997, 0.61540718, 0.62099921, 0.62817333, 0.61802478,
       0.62051535, 0.63165802, 0.64096884, 0.62168573, 0.62852825,
       0.64209086, 0.65189249])

In [74]:
selector.cv_results_['std_test_score']

array([0.0020553 , 0.00312245, 0.00204695, 0.001689  , 0.00335137,
       0.00309493, 0.00289787, 0.00347016, 0.00280734, 0.00203748,
       0.00216056, 0.00271549])

In [79]:
best = selector.best_estimator_

scaler_X = StandardScaler().fit(X_test)
scaled_X = scaler_X.transform(X_test)
scaled_X = pd.DataFrame(scaled_X,index=X_test.index)

predict = best.predict(scaled_X)
probs = best.predict_proba(scaled_X)

In [86]:
roc_auc_score(y_test,probs[:,1])

0.5593911347310342

In [83]:
probs[:,1]

array([0.13716158, 0.04746452, 0.01001592, ..., 0.03384816, 0.02971134,
       0.05862004])